In [1]:
%%capture
import os

# check if this is a colab notebook and clone the repo if it is
if 'COLAB_GPU' in os.environ:
    %cd /content/
    !git clone https://github.com/Enver-group/twitch-web-analytics
    %cd twitch-web-analytics
else:
    os.chdir('..')

!pip install -r requirements.txt

!pip install -e .

In [2]:
#autoreload modules
%load_ext autoreload
%autoreload 2

from src.user import User

In [3]:
ibai = User.from_name("ibai")
ibai_follows = ibai.follows
print(f"{ibai.name.capitalize()} follows {len(ibai_follows)} users")

Ibai follows 155 users


In [4]:
users_that_ibai_follows = User.get_users(ibai_follows)
print("%s users retrieved from the list of follows"%len(users_that_ibai_follows))

155 users retrieved from the list of follows


In [5]:
import pandas as pd
df = pd.DataFrame([ibai]+users_that_ibai_follows)
df

,id,name,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286987200,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"[35980866, 145908612, 205218019, 91136321, 524..."
1,35980866,Perxitaa,partner,Troll de Youtube http://www.youtube.com/user/p...,es,Grand Theft Auto V,38763936,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-09-05T22:12:47Z,None
2,145908612,momoladinastia,partner,"Siempre al pie para los mios, Stream #ATR re b...",es,Just Chatting,11215686,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-01-25T01:52:25Z,None
3,91136321,karchez,partner,"Soy Karchez, me gusta llevar corona y hago el ...",es,Just Chatting,13130932,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-05-15T21:35:29Z,None
4,52495684,Rioboo,partner,Un chavalito calmao. Tengo 10 ideas al día y l...,es,Just Chatting,528710,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-12-01T15:24:26Z,None
...,...,...,...,...,...,...,...,...,...,...
151,50244027,Kuentinlol,partner,,es,Ni no Kuni: Wrath of the White Witch Remastered,2956904,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-10-16T20:05:00Z,None
152,36138196,alexelcapo,partner,Nací en el 87 y me gusta jugar a videojuegos.,es,Minecraft,71825887,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-09-12T21:24:26Z,None
153,29841683,NubbinsGoody,partner,My name is Jonas Neubauer and I'm the 7-time C...,en,,645966,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-04-17T04:28:37Z,None
154,167100879,littleragergirl,partner,"｡･:*:･ﾟ★,｡･:*:･ﾟ☆｡･:*:･ﾟ★,｡･:*:･ﾟ☆｡･:*:･ﾟ★,｡･:...",es,Just Chatting,17375306,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-07-28T10:46:09Z,None


In [6]:
from src.data import make_data_from_root_user

# Obtiene datos de cada usuario del arbol empezando por Ibai y sus follows
# cada iteracion siguiente coge un usuario del arbol aleatorio para añadir sus follows 
# y asi hasta llegar a max_users o que se interrumpa manualmente. 
# Cada 50 iteraciones guarda el progreso actual en un archivo csv
make_data_from_root_user("ibai",output_filepath="data/data.csv",max_users=200)

INFO:src.data.make_dataset:Collecting users from a tree of twitch users follows starting from the follows of the user: ibai
INFO:src.data.make_dataset:New users will be fetched until 500 users are reached or the program is manually interrupted.
INFO:src.data.make_dataset:The tree will be expanded randomly starting from the root.
INFO:src.data.make_dataset:Iteration 1 - 140 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:max_users reached. 565 users were retrieved.
INFO:src.data.make_dataset:writing dataset to file before stopping...


In [12]:
data = pd.read_csv("data/data.csv")
data.drop_duplicates("id", keep='first',inplace=True)
data.fillna({"broadcaster_type":"user"},inplace=True)
data = data.loc[data['lang']=='es']

In [18]:
data.sort_values(by='view_count', ascending=False)

,id,name,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286987200,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"['35980866', '145908612', '205218019', '911363..."
495,22346597,LVPes,partner,NaN,es,Special Events,227606406,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-05-11T10:25:51Z,NaN
182,39276140,Rubius,partner,se contar hasta patata,es,Minecraft,222697739,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-12T18:46:56Z,NaN
106,459331509,auronplay,partner,feliz pero no mucho,es,Minecraft,212058287,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03T14:02:49Z,NaN
149,48878319,TheGrefg,partner,"Hola, me llamo David, me quedé calvo delante d...",es,Fortnite,197647070,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-12T00:45:27Z,NaN
...,...,...,...,...,...,...,...,...,...,...
276,148044248,koreanred,user,NaN,es,League of Legends,558,https://static-cdn.jtvnw.net/user-default-pict...,2017-02-18T00:21:31Z,NaN
383,138286841,bohitigaming,affiliate,NaN,es,Forza Horizon 5,546,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-10-30T04:35:04Z,NaN
472,77276138,Piindaro,user,NaN,es,League of Legends,404,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-12-15T19:06:09Z,NaN
523,161781459,Sasi_Monfreak,user,NaN,es,Overwatch,192,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-06-27T19:06:31Z,NaN


In [16]:
from src.data import extract_follows_from_users_df
extract_follows_from_users_df(data)

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,None,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286987200,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"['35980866', '145908612', '205218019', '911363..."
1,57793021,LOLITOFDEZ,None,partner,| CHEATER 100% CONFIRMED |,es,None,37833154,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-02-26T19:51:46Z,"['222405749', '156467213', '189726839', '21612..."
2,39115143,Th3Antonio,None,partner,EL PELUCAS,es,League of Legends,27273457,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-07T00:28:35Z,"['66024044', '427520929', '58753574', '2555630..."
3,73748275,VicioOneMoreTime,None,partner,None,es,Just Chatting,1578571,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-10-25T20:08:40Z,"['615317640', '535903144', '138286841', '57770..."
4,33734881,Send0o,None,partner,🇩🇪 🇪🇸 - Challenger Toplaner - Business inquiri...,es,Just Chatting,5869231,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-08-14T06:40:58Z,"['38768806', '38951075', '208411643', '4539914..."
...,...,...,...,...,...,...,...,...,...,...,...
557,41766568,Shorbie,None,affiliate,Welcome Welcome! Me encontrareis jugando a LoL...,es,League of Legends,28921,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-03-27T15:23:11Z,"[43330797, 46715307, 157440062, 200716639, 710..."
558,142339574,Movistar_Riders,None,partner,Movistar Riders es un club de eSports creado p...,es,Counter-Strike: Global Offensive,7781141,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-12-19T15:06:33Z,"[31239503, 79005469, 594191118, 21681549, 1079..."
559,163937134,Morphains,None,partner,me gustan los patos,es,Just Chatting,2534255,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-07-10T19:18:51Z,"[675241449, 74101321, 22346597, 40909862, 5703..."
560,127571954,xIsma,None,partner,None,es,Fortnite,1217550,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-06-24T00:24:58Z,"[222705894, 691109063, 42351942, 132279966, 12..."


In [19]:
from src.data import make_data_from_root_user
from src.data import extract_follows_from_users_df

max_users = 100000
top_users = 5000
initial_user = 'ibai'

make_data_from_root_user(initial_user,output_filepath="data/data.csv",max_users=max_users)
data = pd.read_csv("data/data.csv")
data.drop_duplicates("id", keep='first',inplace=True)
data.fillna({"broadcaster_type":"user"},inplace=True)
data = data.loc[data['lang']=='es']
data = data.loc[data['broadcaster_type']!='user']
data.sort_values(by='view_count', ascending=False,inplace=True)

top_df = data.head(top_users).copy()

top_df = extract_follows_from_users_df(top_df)

for i in range(0,top_users):
    data.iloc[i] = top_df.iloc[i]

data.to_csv(r'data/data.csv',index=False,header=True)

INFO:src.data.make_dataset:Collecting users from a tree of twitch users follows starting from the follows of the user: ibai
INFO:src.data.make_dataset:New users will be fetched until 1000 users are reached or the program is manually interrupted.
INFO:src.data.make_dataset:The tree will be expanded randomly starting from the root.
INFO:src.data.make_dataset:Iteration 1 - 140 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:Iteration 11 - 660 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:max_users reached. 1075 users were retrieved.
INFO:src.data.make_dataset:writing dataset to file before stopping...
